In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import json
import os

import analysis.pipelines.population_spikes as ps
from analysis import utils

RESULT_PATH = "simulation_results"
OUTPUT_PATH = "analysis_results"

save_data = True

In [2]:
ps.set_variables(RESULT_PATH=RESULT_PATH, OUTPUT_PATH=OUTPUT_PATH)

PN_pop_names = ps.PN_pop_names
ITN_pop_names = ps.ITN_pop_names

#### Set trials and statistics

In [3]:
filter = ('rand', 'div')
revert_junction = False
exclude = ['baseline', 'short', 'long']

with open(os.path.join(OUTPUT_PATH, 'trials_ordered.json'), 'r') as f:
    trials_ordered = json.load(f)
trials = ps.get_trials(filter, trials=trials_ordered, revert_junction=revert_junction, exclude=exclude)
print(trials)

['ramp_a0_t0.3_rand', 'ramp_a0_t0.3_div', 'ramp_a0_t1.0_rand', 'ramp_a0_t1.0_div', 'ramp_a0_t1.0_down_rand', 'ramp_a0_t1.0_down_div', 'ramp_a0_t3.0_rand', 'ramp_a0_t3.0_div', 'join_a0_t0.3_rand', 'join_a0_t0.3_div', 'join_a0_t1.0_rand', 'join_a0_t1.0_div', 'join_a0_t1.0_quit_rand', 'join_a0_t1.0_quit_div', 'join_a0_t3.0_rand', 'join_a0_t3.0_div', 'fade_a01_t0.1_rand', 'fade_a01_t0.1_div', 'fade_a03_t0.1_div', 'fade_a01_t0.3_rand', 'fade_a01_t0.3_div', 'fade_a03_t0.3_div', 'fade_a01_t1.0_rand', 'fade_a01_t1.0_div', 'fade_a03_t1.0_div', 'fade_a01_t3.0_rand', 'fade_a01_t3.0_div', 'fade_a03_t3.0_div']


In [4]:
wave_pop = 'ITN'

waves = ['gamma', 'beta']
pop_groups = ['PN stimulated', 'PN rest'] + ITN_pop_names
stats = ['mean', 'stdev']
stat_func = {'mean': np.mean, 'stdev': np.std}

# define function for retrieving unit ids in each group
def get_grp_ids(plv_ds):
    ids = {}
    idx = plv_ds.assy_id.values >= 0
    ids['PN stimulated'] = np.concatenate([plv_ds[p].values for p in plv_ds.PN_names[idx].values])
    ids['PN rest'] = np.concatenate([plv_ds[p].values for p in plv_ds.PN_names[~idx].values])
    for p in ITN_pop_names:
        ids[p] = plv_ds[p].values
    return ids

columns = pd.MultiIndex.from_product((waves, pop_groups, stats))
plv_list = []
trial_label = []

#### Get statistics dataframe

In [5]:
significant_duration = True  # whether normalize indicator by baseline, otherwise by trial average
suffix = '_sigdur' if significant_duration else ''

for tr in trials:
    file = os.path.join(ps.FR_ENTR_PATH, tr + suffix + '.nc')
    if not os.path.isfile(file):
        continue
    trial_label.append(utils.get_trial_label(tr, dlm=' '))
    plv_ds = xr.load_dataset(file)
    grp_ids = get_grp_ids(plv_ds)
    plv_stat = pd.Series(0., index=columns)
    for grp, ids in grp_ids.items():
        for w in waves:
            plv = plv_ds.PLV.sel(unit_id=ids, wave=w, wave_population=wave_pop)
            for stat in stats:
                plv_stat.loc[(w, grp, stat)] = stat_func[stat](plv.values)
    plv_list.append(plv_stat.values)

plv_df = pd.DataFrame(plv_list, index=pd.Index(trial_label, name='trial'), columns=columns)

Save to file

In [6]:
display(plv_df)
if save_data:
    file_name = 'PLV_to_' + wave_pop + '_FR' + suffix + '.csv'
    plv_df.to_csv(os.path.join(ps.FR_ENTR_PATH, file_name))

gamma                                          \
                       PN stimulated             PN rest                 FSI   
                                mean     stdev      mean     stdev      mean   
trial                                                                          
ramp random0 t0.3           0.072808  0.120205  0.130059  0.255202  0.166793   
ramp strong0 t0.3           0.074624  0.118936  0.123595  0.248811  0.193610   
ramp random0 t1.0           0.063497  0.107995  0.124706  0.238731  0.152004   
ramp strong0 t1.0           0.057219  0.095168  0.127076  0.240055  0.149628   
ramp down random0 t1.0      0.135692  0.270736  0.121340  0.258608  0.148720   
ramp down strong0 t1.0      0.131568  0.257242  0.121473  0.257163  0.109228   
ramp random0 t3.0           0.054204  0.091920  0.104073  0.198467  0.136045   
ramp strong0 t3.0           0.050681  0.084857  0.103149  0.194767  0.147220   
join random0 t0.3           0.079655  0.174604  0.119964  0.249873  0.148150   
join strong0 t0.3           0.081193  0.178837  0.126251  0.257088  0.164340   
join random0 t1.0           0.097291  0.215040  0.118391  0.248969  0.140767   
join strong0 t1.0           0.103349  0.226029  0.124910  0.261524  0.141550   
quit random0 t1.0           0.098834  0.248741  0.090940  0.239651  0.145194   
quit strong0 t1.0           0.094316  0.246883  0.098079  0.247941  0.147448   
join random0 t3.0           0.111844  0.236275  0.122263  0.249576  0.135572   
join strong0 t3.0           0.127569  0.246835  0.127276  0.257219  0.135401   
fade random01 t0.1          0.095813  0.199265  0.069831  0.220689  0.285536   
fade strong01 t0.1          0.113627  0.205988  0.069349  0.218607  0.339093   
fade strong03 t0.1          0.098853  0.218206  0.060058  0.203589  0.234656   
fade random01 t0.3          0.095140  0.210861  0.057784  0.203244  0.156838   
fade strong01 t0.3          0.113073  0.227389  0.060205  0.203982  0.182508   
fade strong03 t0.3          0.103725  0.231174  0.042261  0.176233  0.120991   
fade random01 t1.0          0.000000  0.000000  0.000000  0.000000  0.000000   
fade strong01 t1.0          0.000000  0.000000  0.000000  0.000000  0.000000   
fade strong03 t1.0          0.000000  0.000000  0.000000  0.000000  0.000000   
fade random01 t3.0          0.000000  0.000000  0.000000  0.000000  0.000000   
fade strong01 t3.0          0.000000  0.000000  0.000000  0.000000  0.000000   
fade strong03 t3.0          0.000000  0.000000  0.000000  0.000000  0.000000   

                                                              beta            \
                                       LTS           PN stimulated             
                           stdev      mean     stdev          mean     stdev   
trial                                                                          
ramp random0 t0.3       0.166412  0.127467  0.195009      0.057332  0.118839   
ramp strong0 t0.3       0.187008  0.153022  0.199068      0.061744  0.119646   
ramp random0 t1.0       0.153180  0.087196  0.165370      0.078262  0.100231   
ramp strong0 t1.0       0.146895  0.081563  0.165844      0.092979  0.108296   
ramp down random0 t1.0  0.228526  0.102026  0.227305      0.123675  0.131166   
ramp down strong0 t1.0  0.197976  0.080942  0.196983      0.136370  0.132023   
ramp random0 t3.0       0.122595  0.075193  0.137139      0.103351  0.085285   
ramp strong0 t3.0       0.124319  0.080008  0.144165      0.119697  0.090378   
join random0 t0.3       0.187156  0.125520  0.218012      0.063717  0.139616   
join strong0 t0.3       0.192418  0.115923  0.178883      0.063593  0.143134   
join random0 t1.0       0.199430  0.096679  0.201265      0.108375  0.143580   
join strong0 t1.0       0.194443  0.105342  0.205626      0.132754  0.150603   
quit random0 t1.0       0.251266  0.084525  0.208050      0.134233  0.157083   
quit strong0 t1.0       0.251675  0.088150  0.217264      0.168208  0.173941   
join random0 t3.0       0.1877

#### Format print statistics

In [7]:
@np.vectorize
def format_stats(mean, std):
    return f'{mean:.3f} ({std:.3f})'

pd_idx = pd.IndexSlice

means = plv_df.loc[:, pd_idx[:, :, 'mean']].values
stds = plv_df.loc[:, pd_idx[:, :, 'stdev']].values
stat_df = pd.DataFrame(format_stats(means, stds), index=plv_df.index,
                       columns=columns.droplevel(2).drop_duplicates())

In [8]:
display(stat_df)

if save_data:
    file_name = 'PLV_to_' + wave_pop + '_FR' + suffix + '_fmt.csv'
    stat_df.to_csv(os.path.join(ps.FR_ENTR_PATH, file_name))

gamma                                \
                        PN stimulated        PN rest            FSI   
trial                                                                 
ramp random0 t0.3       0.073 (0.120)  0.130 (0.255)  0.167 (0.166)   
ramp strong0 t0.3       0.075 (0.119)  0.124 (0.249)  0.194 (0.187)   
ramp random0 t1.0       0.063 (0.108)  0.125 (0.239)  0.152 (0.153)   
ramp strong0 t1.0       0.057 (0.095)  0.127 (0.240)  0.150 (0.147)   
ramp down random0 t1.0  0.136 (0.271)  0.121 (0.259)  0.149 (0.229)   
ramp down strong0 t1.0  0.132 (0.257)  0.121 (0.257)  0.109 (0.198)   
ramp random0 t3.0       0.054 (0.092)  0.104 (0.198)  0.136 (0.123)   
ramp strong0 t3.0       0.051 (0.085)  0.103 (0.195)  0.147 (0.124)   
join random0 t0.3       0.080 (0.175)  0.120 (0.250)  0.148 (0.187)   
join strong0 t0.3       0.081 (0.179)  0.126 (0.257)  0.164 (0.192)   
join random0 t1.0       0.097 (0.215)  0.118 (0.249)  0.141 (0.199)   
join strong0 t1.0       0.103 (0.226)  0.125 (0.262)  0.142 (0.194)   
quit random0 t1.0       0.099 (0.249)  0.091 (0.240)  0.145 (0.251)   
quit strong0 t1.0       0.094 (0.247)  0.098 (0.248)  0.147 (0.252)   
join random0 t3.0       0.112 (0.236)  0.122 (0.250)  0.136 (0.188)   
join strong0 t3.0       0.128 (0.247)  0.127 (0.257)  0.135 (0.197)   
fade random01 t0.1      0.096 (0.199)  0.070 (0.221)  0.286 (0.292)   
fade strong01 t0.1      0.114 (0.206)  0.069 (0.219)  0.339 (0.308)   
fade strong03 t0.1      0.099 (0.218)  0.060 (0.204)  0.235 (0.278)   
fade random01 t0.3      0.095 (0.211)  0.058 (0.203)  0.157 (0.265)   
fade strong01 t0.3      0.113 (0.227)  0.060 (0.204)  0.183 (0.286)   
fade strong03 t0.3      0.104 (0.231)  0.042 (0.176)  0.121 (0.252)   
fade random01 t1.0      0.000 (0.000)  0.000 (0.000)  0.000 (0.000)   
fade strong01 t1.0      0.000 (0.000)  0.000 (0.000)  0.000 (0.000)   
fade strong03 t1.0      0.000 (0.000)  0.000 (0.000)  0.000 (0.000)   
fade random01 t3.0      0.000 (0.000)  0.000 (0.000)  0.000 (0.000)   
fade strong01 t3.0      0.000 (0.000)  0.000 (0.000)  0.000 (0.000)   
fade strong03 t3.0      0.000 (0.000)  0.000 (0.000)  0.000 (0.000)   

                                                beta                 \
                                  LTS  PN stimulated        PN rest   
trial                                                                 
ramp random0 t0.3       0.127 (0.195)  0.057 (0.119)  0.100 (0.248)   
ramp strong0 t0.3       0.153 (0.199)  0.062 (0.120)  0.092 (0.240)   
ramp random0 t1.0       0.087 (0.165)  0.078 (0.100)  0.127 (0.245)   
ramp strong0 t1.0       0.082 (0.166)  0.093 (0.108)  0.126 (0.246)   
ramp down random0 t1.0  0.102 (0.227)  0.124 (0.131)  0.127 (0.240)   
ramp down strong0 t1.0  0.081 (0.197)  0.136 (0.132)  0.120 (0.233)   
ramp random0 t3.0       0.075 (0.137)  0.103 (0.085)  0.110 (0.194)   
ramp strong0 t3.0       0.080 (0.144)  0.120 (0.090)  0.111 (0.196)   
join random0 t0.3       0.126 (0.218)  0.064 (0.140)  0.113 (0.252)   
join strong0 t0.3       0.116 (0.179)  0.064 (0.143)  0.113 (0.257)   
join random0 t1.0       0.097 (0.201)  0.108 (0.144)  0.118 (0.231)   
join strong0 t1.0       0.105 (0.206)  0.133 (0.151)  0.122 (0.235)   
quit random0 t1.0       0.085 (0.208)  0.134 (0.157)  0.127 (0.240)   
quit strong0 t1.0       0.088 (0.217)  0.168 (0.174)  0.121 (0.232)   
join random0 t3.0       0.088 (0.201)  0.138 (0.128)  0.117 (0.200)   
join strong0 t3.0       0.086 (0.192)  0.166 (0.140)  0.122 (0.197)   
fade random01 t0.1      0.227 (0.270)  0.092 (0.177)  0.105 (0.250)   
fade strong01 t0.1      0.274 (0.285)  0.120 (0.199)  0.101 (0.241)   
fade strong03 t0.1      0.181 (0.248)  0.122 (0.196)  0.101 (0.242)   
fade random01 t0.3      0.108 (0.216)  0.098 (0.153)  0.111 (0.242)   
fade strong01 t0.3      0.116 (0.210)  0.105 (0.151)  0.115 (0.247)   
fade strong03 t0.3      0.107 (0.212)  0.108 (0.153)  0.110 (0.244)   
fade random01 t1.0      0.000 (0.000)  0.139 (0.147)  0